# Structure Growth and fσ₈(z)

This notebook computes the linear growth factor **D(z)** and **fσ₈(z)** for the Waveframe V4.0 model,
and compares them to redshift-space distortion (RSD) observational data.

**Goals:**
1. Integrate the growth equation for matter perturbations given a model-specific H(z).
2. Compute fσ₈(z) for direct comparison with current large-scale structure surveys.
3. Evaluate fit quality against ΛCDM.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Example placeholder H(z) function for Waveframe V4.0 (to be replaced with actual model)
def H_waveframe(z):
    H0 = 70.0  # km/s/Mpc
    return H0 * np.sqrt(0.3 * (1+z)**3 + 0.7)

# Growth equation d^2D/d(ln a)^2 + [ (dlnH/dlna) + 2 ] dD/d(ln a) - (3/2)*Omega_m(a) * D = 0
def growth_equation(ln_a, y):
    a = np.exp(ln_a)
    z = 1/a - 1
    H = H_waveframe(z)
    dlnH_dlna = -0.5 * (3*0.3*(1+z)**3) / (0.3*(1+z)**3 + 0.7)
    Omega_m_a = 0.3 * (1+z)**3 / ((H/H_waveframe(0))**2)
    D, dD_dlna = y
    d2D_dlna2 = - (dlnH_dlna + 2) * dD_dlna + 1.5 * Omega_m_a * D
    return [dD_dlna, d2D_dlna2]

# Solve from high redshift to present
a_init, a_final = 1e-3, 1.0
ln_a_span = (np.log(a_init), np.log(a_final))
y0 = [a_init, 1.0]  # Initial D(a) ~ a, dD/dln a ~ 1 at early times
sol = solve_ivp(growth_equation, ln_a_span, y0, dense_output=True)

# Extract results
a_vals = np.exp(np.linspace(np.log(a_init), np.log(a_final), 200))
D_vals = sol.sol(np.log(a_vals))[0]
D_vals /= D_vals[-1]  # Normalize to D(a=1) = 1
z_vals = 1/a_vals - 1

# Compute f = dlnD/dlna and fσ₈(z)
dlnD_dlna = np.gradient(np.log(D_vals), np.log(a_vals))
sigma8_0 = 0.8
fsigma8_vals = dlnD_dlna * sigma8_0 * D_vals

# Plot
plt.figure(figsize=(8,5))
plt.plot(z_vals, fsigma8_vals, label='Waveframe V4.0 (example)')
plt.gca().invert_xaxis()
plt.xlabel('Redshift z')
plt.ylabel('fσ₈(z)')
plt.title('Structure Growth: Waveframe vs Observations')
plt.legend()
plt.grid(True)
plt.show()